<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/Bethelegend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tqdm
import os

In [3]:
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print(os.getcwd())

/content


In [109]:
data = pd.read_excel("/content/날짜별 타자 데이터 통합.xlsx")
hitter = pd.read_excel("/content/90명 타자 명단.xlsx")
vs_avg_data = pd.read_excel("/content/팀 상대 타율 정렬.xlsx")
pitcher_stadium = pd.read_excel("/content/2021시즌 타자 데이터.xlsx")

In [114]:
our_data.head(5)

,day,name,vs,ab,hit,avg,slg,vs_ab,vs_hit,vs_avg,recent_5,recent_10,success_recent_10,recent_5days,result
0,44289,강민호,키움,3,0,0.000,0.000,0.0,0.0,0.0,0.000000,0.000,0.00,0.000000,0
1,44290,강민호,키움,4,2,0.286,0.286,3.0,0.0,0.0,0.000000,0.000,0.00,0.000000,1
2,44292,강민호,두산,1,0,0.250,0.250,0.0,0.0,0.0,0.285714,0.286,0.50,0.285714,0
3,44293,강민호,두산,4,1,0.250,0.333,1.0,0.0,0.0,0.250000,0.250,0.33,0.250000,1
4,44294,강민호,두산,3,2,0.333,0.667,5.0,1.0,0.2,0.250000,0.250,0.50,0.250000,1


In [110]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)
vs_avg_data = pd.DataFrame(vs_avg_data)
pitcher_stadium = pd.DataFrame(pitcher_stadium)

In [115]:
#일단 이 함수는 나중에,,
def make_pitcher_stadium(record_data, pitcher_stadium):
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  vs = np.empty(len(record_data), dtype=np.chararray)
  over
  stadium_avg = np.zeros(len(record_data))
  
  
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))

In [ ]:
print(data.head(), len(data))

In [ ]:
print(hitter.head(), len(hitter))

In [ ]:
data.loc[:,['day','name','ab','hit']].dtypes

In [ ]:
data.loc[:,['day','name','ab','hit']]

In [90]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, recent_day=5):

  # 최근 5일 경기 타율 저장할 공간 생성
  recent_5days = np.zeros(len(record_data)) # 최근 5일 경기 타율
  recent_5days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_5days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r in hdata['day']: # 추출한 데이터에서 한 행씩 접근
      recent_5days_hitter[ps] = h # 현재 타자
      recent_5days_day[ps] = r  # 현재 기준 날짜
      hrdata = hdata.loc[hdata['day']>=r-recent_day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기 
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_5days[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = recent_5days_day
  result_Dataframe['name'] = recent_5days_hitter
  result_Dataframe['recent_5days'] = recent_5days
  result_Dataframe

  return result_Dataframe

In [62]:
#상대 타율 구하는 함수
#데이터 파일에서 vs 내림차순 후, 누적 타수 / 누적 안타 계산이 끝난 상태여야함
def make_vs_avg(hitter_data, record_data):
  vs_avg = np.zeros(len(record_data))
  name = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  vs = np.empty(len(record_data), dtype=np.chararray)
  vs_ab = np.zeros(len(record_data))
  vs_hit = np.zeros(len(record_data))

  ps = 0
  for h in tqdm.tqdm(hitter_data['batter']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h] # 특정 타자 한명에 대해서만 데이터 추출
    for r, v,ab, hit in zip(hdata['day'], hdata['vs'], hdata['vs_cu_ab'], hdata['vs_cu_hit']): # 추출한 데이터에서 한 행씩 접근
      day[ps] = r  # 현재 기준 날짜
      name[ps] = h # 현재 타자
      vs[ps] = v
      vs_ab[ps] = ab
      vs_hit[ps] = hit
      try:
        vs_avg[ps] = hit / ab
      except ZeroDivisionError:
        vs_avg[ps] = 0
      ps+=1
  
  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = day
  result_Dataframe['name'] = name
  result_Dataframe['vs_team'] = vs
  result_Dataframe['vs_ab'] = vs_ab
  result_Dataframe['vs_hit'] = vs_hit
  result_Dataframe['vs_avg'] = vs_avg
  result_Dataframe

  return result_Dataframe



In [63]:
vs_avg = make_vs_avg(hitter, vs_avg_data)

100%|██████████| 90/90 [00:00<00:00, 896.72it/s]


In [ ]:
vs_avg.loc[vs_avg['name']=='전준우']

In [91]:
recent_days = make_recent_days(hitter, data)

100%|██████████| 90/90 [00:08<00:00, 10.58it/s]


In [ ]:
recent_days

In [98]:
merge_data1 = pd.merge(data, recent_days, on = ['day', 'name'], how = 'inner')

In [ ]:
merge_data1.drop_duplicates()

In [104]:
len(data), len(recent_days), len(vs_avg), len(merge_data1)

(5296, 5296, 5296, 5586)

In [ ]:
merge_data1.loc[merge_data1['name']=='전준우']

In [ ]:
vs_avg

In [67]:
merge_data2 = merge_data1.merge(vs_avg, on = ['day', 'name'])

In [68]:
our_data = merge_data2.loc[:,['day','name', 'vs', 'ab','hit', 'avg', 'slg', 'vs_ab', 'vs_hit', 'vs_avg', 'recent_5','recent_10','success_recent_10','recent_5days', 'result']]

In [ ]:
our_data.loc[our_data['name']=='전준우']